In [1]:
from adabas.api import *
from adabas.datamap import *
a=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)
a.cb.dbid=12              
a.cb.fnr=11 

In [ ]:
#campo elementar de um grupo periódico ou campo elementar múltiplo
def get_periodic(isn = 0   # isn a ser pesquisado
                ,gsn = ''  # grupo short name ou elementar multiplo    - FDT
                ,esn = ''  # elementar short name - FDT
                ,lgn = ''  # elementar long-name  - DDM
                ,lsn = 0   # length elementar     - DDM
                ,upk = ''  # unpack para colunas packeadas '' or True
                ,scl = ''  # scale or ''          - DDM
                ,Int = ''  # True para numéricos
                 ):
    ret = ''
    try:
        upk = ',U' if upk else ''
        Str = '"'
        a.cb.isn=isn                 
        if scl or Int:
            Int = 'int('
            Str = ''
            if scl:
                scl = ') * .{0:>0{1}}'.format('1', scl)
            else:
                scl = ')'

        a.fb.value='{}C,002,B.'.format(gsn)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            occ=int(a.rb.value.encode('hex') ) # count occurs group
            exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
            row.buffer = a.rb
            a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
            a.get(isn=a.cb.isn)
            ret = eval("""'"{}": [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))    
            ret = ret.replace(', ]', ']')
    except DatabaseError, (error,apa):
        return error.split(':')[1]
    return ret

In [ ]:
#campo multiplo de um grupo periódico
def get_per_mult(isn = 0   # isn a ser pesquisado
                ,gsn = ''  # grupo short name     - FDT
                ,esn = ''  # elementar short name - FDT
                ,lgn = ''  # elementar long-name  - DDM
                ,lsn = 0   # length elementar     - DDM
                ,upk = ''  # unpack para colunas packeadas '' or True
                ,scl = ''  # scale or ''          - DDM
                ,Int = ''  # True para numéricos
                 ):
    ret = ''
    try:
        upk = ',U' if upk else ''
        Str = '"'
        a.cb.isn=isn
        if scl or Int:
            Int = 'int('
            Str = ''
            if scl:
                scl = ') * .{0:>0{1}}'.format('1', scl)
            else:
                scl = ')'
        lst = '['        
        a.fb.value='{}C,002,B.'.format(gsn)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            gocc=int(a.rb.value.encode('hex') ) # count occurs group
            for o in range(1,gocc+1):
                a.fb.value='{}{}C,002,B.'.format(esn,o)
                a.get(isn=a.cb.isn)
                if a.rb.value:
                    occ=int(a.rb.value.encode('hex') ) # count occurs elementar
                    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
                    row.buffer = a.rb
                    a.fb.value='{}{}(1-{}),{}{}.'.format(esn, o, occ, lsn, upk)
                    a.get(isn=a.cb.isn)
                    ret = eval("""'[{}]'.format({})""".format((Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
                    lst += '{}, '.format(ret)
                else:
                    break
    except DatabaseError, (error,apa):
        return error.split(':')[1]
    return '"{}": {}]'.format(lgn, lst).replace(', ]', ']')          

In [ ]:
#campo elementar de um grupo periódico
a.cb.isn=1
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AR'        # elementar short name - FDT
lgn = 'curr_code' # elementar long-name  - DDM
lsn = 3          # length elementar     - DDM
upk = ''         # unpack para colunas packeadas
scl = ''         # scale
Int = ''
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'

a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    ret = eval("""'"{}": [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))    
    print ret
    
print get_periodic(isn=1, gsn = 'AQ', esn = 'AR', lgn = 'curr_code', lsn = 3)

In [ ]:
#campo elementar de um grupo periódico numerico 
a.cb.isn=201
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AS'        # elementar short name - FDT
lgn = 'salary'    # elementar long-name  - DDM
lsn = 9           # length elementar     - DDM
upk = ',U'        # unpack para colunas packeadas ou binary
scl = ''          # scale or ''
Int = True        # True or ''
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'
        
a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    ret = eval("""'"{}": [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))    
    print ret
print get_periodic(isn=201, gsn = 'AQ', esn = 'AS', lgn = 'salary', lsn = 9, upk=True, Int=True)

In [ ]:
#campo elementar múltiplo
a.cb.isn=283
esn = 'AI'           # elementar short name - FDT
lgn = 'address_line' # elementar long-name  - DDM
lsn = 20             # length elementar     - DDM
upk = ''             # unpack para colunas packeadas
scl = ''             # scale or ''
Int = ''             # True or ''  
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'

a.fb.value='{}C,002,B.'.format(esn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    ret =  eval("""'"{}": [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))    
    print ret
print get_periodic(isn=283, gsn = 'AI', esn = 'AI', lgn = 'address_line', lsn = 20)

In [ ]:
#campo multiplo de um grupo periódico
a.cb.isn=283
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AT'        # elementar short name - FDT
lgn = 'bonus'     # elementar long-name  - DDM
lsn = 9           # length elementar     - DDM
upk = ',U'        # unpack para colunas packeadas
scl = ''          # scale or ''
Int = True
Str = '"'
if scl or Int:
    Int = 'int('
    Str = ''
    if scl:
        scl = ') * .{0:>0{1}}'.format('1', scl)
    else:
        scl = ')'
lst = []        
a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    gocc=int(a.rb.value.encode('hex') ) # count occurs group
    for o in range(1,gocc+1):
        a.fb.value='{}{}C,002,B.'.format(esn,o)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            occ=int(a.rb.value.encode('hex') ) # count occurs elementar
            exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
            print '{:08} - occurs {} times'.format(a.cb.isn, occ)
            row.buffer = a.rb
            a.fb.value='{}{}(1-{}),{}{}.'.format(esn, o, occ, lsn, upk)
            a.get(isn=a.cb.isn)
            ret = eval("""'[{}]'.format({})""".format((Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
            print ret            
            lst.append(eval(ret))
        else:
            break
print '"{}": {}'.format(lgn, lst) 
print get_per_mult(isn=283, gsn='AQ', esn='AT', lgn='bonus', lsn=9, upk=True, Int=True)

In [ ]:
a.cb.isn=283
a.fb.value='AQC,2,B.'
a.get(isn=a.cb.isn)
a.rb.value.encode('hex')

In [ ]:
a.fb.value='AT1C,2,B.'
a.get(isn=a.cb.isn)
occ = a.rb.value.encode('hex')
occ

In [ ]:
a.fb.value='AT1(1-{}),9,U.'.format(occ)
a.get(isn=a.cb.isn)
a.rb.value

In [2]:
a.cb.isn=0
try:
    for c in xrange(2000):
        a.fb.value='AQC,002,B.'
        a.readByIsn(getnext=1)
        if a.rb.value:
            aqc=int(a.rb.value.encode('hex') )
            aql=9
            aqu=',9,U'
            exec ("row = Datamap('Rows', {})".format("String('as{}', {l})," * aqc).format(*range(1,aqc+1), l=aql))
            print '{:08} - occurs {} times'.format(a.cb.isn, aqc)
            row.buffer = a.rb
            a.fb.value='AS1-{}{}.'.format(int(aqc), aqu) 
            a.get(isn=a.cb.isn)
            print eval("'{:08} {}'.format({})".format(a.cb.isn, '- {} ' * aqc, ('row.as{}, '* aqc).format(*range(1,aqc+1))))
    print '{} - registros lidos'.format(c)
except DataEnd:
    print 'acabou o file'
except DatabaseError, (error,apa):
    print 'saiu por aqui {} - registros lidos'.format(c), error

00000001 - occurs 1 times
00000001 - 000000963 
00000002 - occurs 1 times
00000002 - 000000998 
00000003 - occurs 1 times
00000003 - 000001036 
00000004 - occurs 1 times
00000004 - 000001005 
00000005 - occurs 1 times
00000005 - 000001008 
00000006 - occurs 1 times
00000006 - 000001024 
00000007 - occurs 1 times
00000007 - 000000987 
00000008 - occurs 1 times
00000008 - 000000962 
00000009 - occurs 1 times
00000009 - 000001023 
00000010 - occurs 1 times
00000010 - 000001057 
00000011 - occurs 1 times
00000011 - 000001036 
00000012 - occurs 1 times
00000012 - 000001132 
00000013 - occurs 1 times
00000013 - 000002203 
00000014 - occurs 1 times
00000014 - 000001001 
00000015 - occurs 1 times
00000015 - 000001054 
00000016 - occurs 1 times
00000016 - 000001027 
00000017 - occurs 1 times
00000017 - 000001016 
00000018 - occurs 1 times
00000018 - 000000938 
00000019 - occurs 1 times
00000019 - 000000999 
00000020 - occurs 1 times
00000020 - 000000975 
00000021 - occurs 1 times
00000021 - 000